In [14]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt

%matplotlib inline

In [15]:
X_train, Y_train = read_csv('train_emoji.csv')
X_test, Y_test = read_csv('tesss.csv')
maxLen = len(max(X_train, key=len).split())

In [16]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.100d.txt')

In [17]:

def sentences_to_indices(X, word_to_index, max_len):
   
    m = X.shape[0]                                   
    
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                              
        
        sentence_words =X[i].lower().split()
        
        j = 0
        
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j+1

    
    return X_indices

In [18]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices = [[ 155345.  225122.       0.       0.       0.]
 [ 220930.  286375.   69714.       0.       0.]
 [ 151204.  192973.  302254.  151349.  394475.]]


In [19]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["cucumber"].shape[0]     
    
    emb_matrix =np.zeros((vocab_len,emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)


    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer


In [20]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -1.5434


In [21]:

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    
    
   
    sentence_indices = Input(input_shape, dtype = 'int32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)  
    
    X = LSTM(128, return_sequences = True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences = False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    model = Model(inputs = sentence_indices, outputs = X)
    
    return model

In [22]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 10, 100)           40000100  
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           117248    
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
__________

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [25]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 1s 6ms/step - loss: 1.6020 - acc: 0.2348
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5416 - acc: 0.3333
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4868 - acc: 0.3182
Epoch 4/50
132/132 [==============================] - 0s 2ms/step - loss: 1.4187 - acc: 0.3939
Epoch 5/50
132/132 [==============================] - 0s 2ms/step - loss: 1.2877 - acc: 0.4773
Epoch 6/50
132/132 [==============================] - 0s 1ms/step - loss: 1.1666 - acc: 0.5530
Epoch 7/50
132/132 [==============================] - 0s 2ms/step - loss: 1.0960 - acc: 0.5303
Epoch 8/50
132/132 [==============================] - 0s 1ms/step - loss: 0.9745 - acc: 0.6136
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 0.8119 - acc: 0.7348
Epoch 10/50
132/132 [==============================] - 0s 2ms/step - loss: 0.7150 - acc: 0.7727
Epoch 11/50
132/132 [============================

In [26]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 4ms/step

Test accuracy =  0.839285722801


In [27]:
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he got a very nice raise	❤️
Expected emoji:😄 prediction: she got me a nice present	❤️
Expected emoji:😄 prediction: he is a good friend	❤️
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😄 prediction: Congratulation for having a baby	❤️
Expected emoji:😄 prediction: you brighten my day	❤️
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😄 prediction: will you be my valentine	❤️
Expected emoji:😞 prediction: go away	⚾


In [28]:
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

not feeling happy 😞
